<a href="https://colab.research.google.com/github/ShabnaIlmi/Data-Science-Group-Project/blob/Future_Risk_prediction/DSGP_with_K_mean%2BDBSCAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
from sklearn.impute import KNNImputer

In [4]:
#mmount google drive with colab
drive.mount('/content/drive')

#loading the Dataset

df = pd.read_excel('/content/drive/MyDrive/DSGP/IMPORT STATISTICS - 2023.xlsx')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Displaying the dataset

df.head()

,IMPORTER,YEAR,MONTH,HSCODE,COUNTRY,UNIT,QUANTITY,VALUE_RS,DESCRIPTION_01,DESCRIPTION_02,DESCRIPTION_03
0,Unknown,2023.0,JANUARY,280110,BANGLADESH,Kg.,131600.0,13877114.0,Inorganic chemicals; Organic or inorganic comp...,"Fluorine, chlorine, bromine and iodine ...",Chlorine
1,Unknown,2023.0,JANUARY,280110,INDIA,Kg.,37800.0,3924989.0,Inorganic chemicals; Organic or inorganic comp...,"Fluorine, chlorine, bromine and iodine ...",Chlorine
2,Unknown,2023.0,JANUARY,280110,MALAYSIA,Kg.,12090.0,5390772.0,Inorganic chemicals; Organic or inorganic comp...,"Fluorine, chlorine, bromine and iodine ...",Chlorine
3,Unknown,2023.0,FEBRUARY,280110,BANGLADESH,Kg.,25200.0,3089445.0,Inorganic chemicals; Organic or inorganic comp...,"Fluorine, chlorine, bromine and iodine ...",Chlorine
4,Unknown,2023.0,FEBRUARY,280110,INDIA,Kg.,75599.0,6939860.0,Inorganic chemicals; Organic or inorganic comp...,"Fluorine, chlorine, bromine and iodine ...",Chlorine


In [6]:
#displaying the dataset information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1042 entries, 0 to 1041
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   IMPORTER        1042 non-null   object 
 1   YEAR            292 non-null    float64
 2   MONTH           292 non-null    object 
 3   HSCODE          1042 non-null   int64  
 4   COUNTRY         1038 non-null   object 
 5   UNIT            1042 non-null   object 
 6   QUANTITY        1039 non-null   float64
 7   VALUE_RS        292 non-null    float64
 8   DESCRIPTION_01  1042 non-null   object 
 9   DESCRIPTION_02  1042 non-null   object 
 10  DESCRIPTION_03  1042 non-null   object 
dtypes: float64(3), int64(1), object(7)
memory usage: 89.7+ KB


In [8]:
#Displaying the statistical summary of dataset
df.describe()

,YEAR,HSCODE,QUANTITY,VALUE_RS
count,292.000000,1.042000e+03,1.039000e+03,2.920000e+02
mean,2018.684932,9.093611e+08,4.241679e+04,3.693244e+06
std,3.409615,1.326227e+09,2.352492e+05,7.946637e+06
min,2016.000000,2.801100e+05,0.000000e+00,0.000000e+00
25%,2016.000000,2.808000e+07,3.500000e+01,1.900250e+01
50%,2016.000000,2.829110e+07,1.000000e+03,1.658700e+02
75%,2023.000000,2.801200e+09,1.989000e+04,4.316572e+06
max,2023.000000,3.102909e+09,4.000000e+06,4.727172e+07


In [9]:
# Identifying categorical and numerical columns
categorical_cols = df.select_dtypes(include=['object']).columns
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns

# Displaying the Categorical and the Numerical columns'
print("Categorical Columns:")
print(categorical_cols)

print("\nNumerical Columns:")
print(numerical_cols)

Categorical Columns:
Index(['IMPORTER', 'MONTH', 'COUNTRY', 'UNIT', 'DESCRIPTION_01',
       'DESCRIPTION_02', 'DESCRIPTION_03'],
      dtype='object')

Numerical Columns:
Index(['YEAR', 'HSCODE', 'QUANTITY', 'VALUE_RS'], dtype='object')


In [11]:
# Converting 'IMPORTER' and 'HSCODE' columns to string type
df['HSCODE'] = df['HSCODE'].astype(str)
df['IMPORTER'] = df['IMPORTER'].astype(str)

# Displaying the HSCODE column data type
print("Data Type of HSCODE Columns:")
print(df[['HSCODE']].dtypes)

print("Data Type of IMPORTER Columns:")
print(df[['IMPORTER']].dtypes)

Data Type of HSCODE Columns:
HSCODE    object
dtype: object
Data Type of IMPORTER Columns:
IMPORTER    object
dtype: object


In [13]:
categorical_cols = df.select_dtypes(include=['object']).columns

# Displaying the categorical features
print("Categorical Features:")
for feature in categorical_cols:
    print(f"- {feature}")

# Display data type of the columns
print("\nData Type of Categorical Features:")
print(df[categorical_cols].dtypes)

Categorical Features:
- IMPORTER
- MONTH
- HSCODE
- COUNTRY
- UNIT
- DESCRIPTION_01
- DESCRIPTION_02
- DESCRIPTION_03

Data Type of Categorical Features:
IMPORTER          object
MONTH             object
HSCODE            object
COUNTRY           object
UNIT              object
DESCRIPTION_01    object
DESCRIPTION_02    object
DESCRIPTION_03    object
dtype: object


In [14]:
# Displaying the unique values and their counts relevant to each categorical column
print("Unique values and their count relevant to each categorical column:\n")
for col in categorical_cols:
    unique_values = df[col].unique()
    value_counts = df[col].value_counts()
    print(value_counts)
    print(" ")

Unique values and their count relevant to each categorical column:

IMPORTER
Unknown    1042
Name: count, dtype: int64
 
MONTH
NOVEMBER     191
FEBRUARY      13
JULY          13
MAY           11
SEPTEMBER     11
JANUARY       10
JUNE           9
AUGUST         8
OCTOBER        8
MARCH          6
APRIL          6
DECEMBER       6
Name: count, dtype: int64
 
HSCODE
28070010      85
2847000000    84
28080000      67
3102401000    58
28291100      54
              ..
2801200032     1
2801200031     1
2801200030     1
2801200029     1
2812101100     1
Name: count, Length: 131, dtype: int64
 
COUNTRY
Vietnam    299
Russia     109
Kenya       71
Ukraine     58
germany     51
          ... 
U.S.A.       1
Mexico       1
CANADA       1
India        1
Romania      1
Name: count, Length: 66, dtype: int64
 
UNIT
Kg     750
Kg.    112
KGS     47
PCS     39
NOS     33
GMS     23
MTS     19
LTR     18
DRM      1
Name: count, dtype: int64
 
DESCRIPTION_01
Inorganic chemicals; Organic or inorganic comp

In [16]:

# Displaying the categorical columns which contains null values and their counts
print("Categorical columns with null values and their counts:")
for col in categorical_cols:
    null_count = df[col].isnull().sum()
    if null_count > 0:
        print(f"{col}: {null_count}")

Categorical columns with null values and their counts:
MONTH: 750
COUNTRY: 4


In [17]:
# Displaying the categorical columns which contain 'Unknown' values and their relevant counts
print("Categorical columns with 'Unknown' values and their counts:")
for col in categorical_cols:
    unknown_count = (df[col] == 'Unknown').sum()
    if unknown_count > 0:
        print(f"{col}: {unknown_count}")

Categorical columns with 'Unknown' values and their counts:
IMPORTER: 1042


In [19]:
# List of numerical features
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns

# Displaying the numerical features
print("Numerical Features:")
for feature in numerical_cols:
    print(f"- {feature}")

# Display the data type of the numerical features
print("\nData Type of Numerical Features:")
print(df[numerical_cols].dtypes)

Numerical Features:
- YEAR
- QUANTITY
- VALUE_RS

Data Type of Numerical Features:
YEAR        float64
QUANTITY    float64
VALUE_RS    float64
dtype: object


In [20]:
# Displaying the unique values and their count in the numerical columns
print("Unique values and their count in the numerical columns:\n")
for col in numerical_cols:
    unique_values = df[col].unique()
    value_counts = df[col].value_counts()
    print(value_counts)
    print(" ")

Unique values and their count in the numerical columns:

YEAR
2016.0    180
2023.0    112
Name: count, dtype: int64
 
QUANTITY
1000.0     55
1.0        49
16200.0    40
5.0        31
27.0       28
           ..
204.0       1
23200.0     1
22820.0     1
22560.0     1
19050.0     1
Name: count, Length: 359, dtype: int64
 
VALUE_RS
0.40           4
53.79          3
188.55         3
0.20           3
21.15          2
              ..
350598.00      1
2213950.00     1
10318355.00    1
24988179.00    1
0.68           1
Name: count, Length: 263, dtype: int64
 


In [21]:
# Displaying the numerical columns with null values and their relevant counts
print("Numerical columns with null values and their relevant counts:")
for col in numerical_cols:
    null_count = df[col].isnull().sum()
    if null_count > 0:
        print(f"{col}: {null_count}")


Numerical columns with null values and their relevant counts:
YEAR: 750
QUANTITY: 3
VALUE_RS: 750


## Data Preprocessing

In [22]:
# Step 1: Removing whitespaces from the object type columns
object_columns = df.select_dtypes(include=['object']).columns
df[object_columns] = df[object_columns].apply(lambda x: x.str.strip())


In [23]:
# Step 2: Removing unnecessary full stops(".") from the categorical columns
df[categorical_cols] = df[categorical_cols].apply(lambda x: x.str.replace('.', ''))

In [24]:
# Define the average weight of one piece for PCS and NOS (update as needed)
average_piece_weight_kg = 1  # Default assumption (can be changed based on item type)

# Conversion mapping based on provided details
unit_conversion = {
    'KG': 1,           # Kg is already in Kg
    'KGS': 1,          # KGS is the same as Kg
    'Kg.': 1,          # Kg. is same as Kg
    'MTS': 1000,       # 1 Metric Ton (MTS) = 1000 Kg
    'GMS': 0.001,      # 1 Gram (GMS) = 0.001 Kg
    'LTR': 1,          # Assume 1 LTR = 1 Kg (change this for specific chemicals)
    'DRM': 200,        # Assuming 1 Drum (DRM) = 200 Kg (can be adjusted)
}

# Function to convert quantities to Kg and update UNIT and QUANTITY columns
def convert_and_update(row):
    unit = row['UNIT'].strip().upper()  # Standardize unit names

    # Handle Piece-based units (PCS, NOS)
    if unit in ['PCS', 'NOS']:
        row['UNIT'] = 'KG'  # Change unit to Kg
        row['QUANTITY'] = row['QUANTITY'] * average_piece_weight_kg  # Convert quantity to Kg

    # Handle other units by conversion factors
    elif unit in unit_conversion:
        row['UNIT'] = 'KG'  # Change unit to Kg
        row['QUANTITY'] = row['QUANTITY'] * unit_conversion[unit]  # Convert quantity to Kg

    return row

# Apply the conversion and update the dataset
df = df.apply(convert_and_update, axis=1)

# Check the resulting dataframe
print(df[['UNIT', 'QUANTITY']].head())


  UNIT  QUANTITY
0   KG  131600.0
1   KG   37800.0
2   KG   12090.0
3   KG   25200.0
4   KG   75599.0


In [27]:
# Step 3: Converting all the values in the 'COUNTRY' and the 'UNIT' columns to uppercase values
df['COUNTRY'] = df['COUNTRY'].str.upper()
df['UNIT'] = df['UNIT'].str.upper()

# Displaying the modified dataset
print(df)

     IMPORTER    YEAR     MONTH      HSCODE      COUNTRY UNIT  QUANTITY  \
0     Unknown  2023.0   JANUARY      280110   BANGLADESH   KG  131600.0   
1     Unknown  2023.0   JANUARY      280110        INDIA   KG   37800.0   
2     Unknown  2023.0   JANUARY      280110     MALAYSIA   KG   12090.0   
3     Unknown  2023.0  FEBRUARY      280110   BANGLADESH   KG   25200.0   
4     Unknown  2023.0  FEBRUARY      280110        INDIA   KG   75599.0   
...       ...     ...       ...         ...          ...  ...       ...   
1037  Unknown     NaN       NaN    28121047  SOUTH KOREA   KG   16200.0   
1038  Unknown     NaN       NaN    28121047  SOUTH KOREA   KG   16200.0   
1039  Unknown     NaN       NaN  2812109900      UKRAINE   KG       NaN   
1040  Unknown     NaN       NaN  2812109900      UKRAINE   KG       NaN   
1041  Unknown     NaN       NaN    28121047  SOUTH KOREA   KG   16200.0   

        VALUE_RS                                     DESCRIPTION_01  \
0     13877114.0  Inorganic 

In [28]:
# handling missing values

# Fill missing numerical columns with the median
df['QUANTITY'].fillna(df['QUANTITY'].median(), inplace=True)

# Fill missing categorical column with 'Unknown'
df['COUNTRY'].fillna('Unknown', inplace=True)


<ipython-input-28-bdf20a6fffbe>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['QUANTITY'].fillna(df['QUANTITY'].median(), inplace=True)
<ipython-input-28-bdf20a6fffbe>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True

In [33]:
# Initialize KNNImputer
knn_imputer = KNNImputer(n_neighbors=5)

# Impute missing numerical columns (YEAR, VALUE_RS)
numerical_cols = ['YEAR', 'VALUE_RS']

# Apply KNN imputation to numerical columns
df[numerical_cols] = knn_imputer.fit_transform(df[numerical_cols])



print(df[['YEAR', 'VALUE_RS']].head())

ValueError: could not convert string to float: 'JANUARY'